In [2]:
include("../../functions/GreedyRegression.jl")
include("../../functions/Integrate.jl")
include("../../functions/ReportModel.jl")
include("../../functions/EnvelopePol.jl")
include("../../functions/PickSubdomains.jl")

using .GreedyRegression, .Integrate, .ReportModel, .EnvelopePol, .PickSubdomains, LinearAlgebra, HDF5, JSON, Pkg

data_path = "../../sims/mhd/mhd_rk4_naive_result.hdf5"
options_path = "settings/spider_settings.json"

Pkg.status()

Status `~/Documents/School/Research/SPIDER_Julia/Project.toml`
  [f67ccb44] HDF5 v0.17.2
  [682c06a0] JSON v1.3.0


In [4]:
json_str::String = read("settings/spider_settings.json", String)
options = JSON.parse(json_str)

fields::Dict = Dict()

h5open("../../sims/mhd/mhd_rk4_naive_result.hdf5", "r") do file
    for (key::String, value::String) in options["fields"]
        fields[key] = read(file, value)
    end
end

fields

Dict{Any, Any} with 8 entries:
  "v"  => [3.02562e-16 3.00037e-16 … 2.91699e-16 3.00232e-16; -0.0978991 -0.097…
  "By" => [-2.93305e-16 0.0490677 … -0.0980171 -0.0490677; -6.94566e-16 0.04900…
  "t"  => [0.0 0.1 … 9.8 9.9]
  "x"  => [0.0 0.0490874 … 6.18501 6.2341]
  "Bx" => [5.17264e-16 2.30693e-16 … 3.08405e-16 2.53142e-16; -0.0490677 -0.049…
  "u"  => [-2.96761e-16 0.0978991 … -0.194151 -0.0978991; -2.92165e-16 0.097075…
  "p"  => [0.9 0.894002 … 0.876312 0.894002; 0.894002 0.888017 … 0.870369 0.888…
  "y"  => [0.0 0.0490874 … 6.18501 6.2341]

In [5]:
nl::Int64 = options["number_of_library_terms"]
nw::Int64 = options["number_of_windows"]
dof::Int64 = options["degrees_of_freedom"]
dim::Int64 = options["dimensions"]
env_pow::Int64 = options["envelope_power"]
size_of_data::Vector{Int} = options["size_of_data"]
size_vec::Vector{Int64} = options["integration_size"]
buffer::Int64 = options["buffer"]
seed::Int64 = options["seed"]

pol::Matrix{Float64} = EnvelopePol.envelope_pol(env_pow, dim)
corners::Matrix{Int} = PickSubdomains.pick_subdomains(size_of_data, size_vec, buffer, nw, seed)

grid = [vec(fields[i]) for i in options["grid"]]

3-element Vector{Vector{Float64}}:
 [0.0, 0.04908738521234052, 0.09817477042468103, 0.14726215563702155, 0.19634954084936207, 0.2454369260617026, 0.2945243112740431, 0.3436116964863836, 0.39269908169872414, 0.44178646691106466  …  5.792311455056181, 5.841398840268521, 5.890486225480862, 5.939573610693203, 5.988660995905543, 6.037748381117884, 6.086835766330224, 6.135923151542564, 6.1850105367549055, 6.234097921967246]
 [0.0, 0.04908738521234052, 0.09817477042468103, 0.14726215563702155, 0.19634954084936207, 0.2454369260617026, 0.2945243112740431, 0.3436116964863836, 0.39269908169872414, 0.44178646691106466  …  5.792311455056181, 5.841398840268521, 5.890486225480862, 5.939573610693203, 5.988660995905543, 6.037748381117884, 6.086835766330224, 6.135923151542564, 6.1850105367549055, 6.234097921967246]
 [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.8999999999999999  …  9.0, 9.1, 9.200000000000001, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]

In [6]:
G::Matrix{Float64} = zeros(Float64, dof*nw, 0)
labels::Vector{String} = String[]
scales::Vector{Float64} = Float64[]

function add_library_term(label::AbstractString)
    term::Dict = options["library_terms"][label]
    vars::Vector{String} = term["vars"]
    operations::Vector{String} = term["operations"]
    derivs::Vector{Int} = term["derivs"]

    index_str(x) = "fields[\""*x*"\"]"
    expr_str = ""
    for i ∈ 1:length(vars)
        expr_str *= operations[i] * index_str(vars[i])
    end

    col::Vector{Float64} = Integrate.integrate(eval(Meta.parse(expr_str)), derivs, grid, corners, size_vec, pol)
    global G = hcat(G, col)
    push!(labels, label)
    push!(scales, term["scale"])
end

for (key, value) in options["library_terms"]
    add_library_term(key)
end

G

128×2 Matrix{Float64}:
 -0.0115984    0.568941
 -0.0581575   -0.455282
 -0.455061    -0.433006
 -0.217499     0.313185
 -0.13585     -0.449276
 -0.309191    -0.0695466
 -0.109318     0.0813084
 -0.551963    -0.0172157
  0.0605835   -0.0916495
 -0.146954     0.792067
  ⋮           
 -0.00968686  -0.0459934
 -0.363277     0.398807
  0.0330666   -0.44469
 -0.0953383    0.108132
 -0.0109284    0.674905
 -0.178094     0.467271
  0.117458    -0.0245697
 -0.13402      0.189473
  0.474377    -0.0976188